<a href="https://colab.research.google.com/github/linshaochieh2019/test/blob/main/rsnaPre_bbox_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install torchio
#!pip install pytorch-lightning

In [ ]:
# Mount to Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torchio as tio
import numpy as np
import pytorch_lightning as pl
from torch.utils.data import random_split, DataLoader, Dataset

class RSNAPre(pl.LightningDataModule):
  def __init__(self, data_dir, batch_size:int=8):
    super().__init__()
    self.data_dir = data_dir
    self.batch_size = batch_size
  
    self.transform = tio.Compose([
                                  tio.transforms.Resize((64, 64, 64)),
                                  ])
    
    self.transform_phase_2 = tio.Compose([
                                          #tio.transforms.ZNormalization(),
                                          tio.RescaleIntensity(out_min_max=(0, 1))
                                          ]) #for after getting bbox

  def setup(self, min_area_threshold=64*64*0.1):
    self.dataset = tio.datasets.RSNAMICCAI(self.data_dir, 
                                          train=self.train, 
                                          transform=self.transform,
                                          )

    self.ls_new_subjects = []
    for patient_index in range(len(self.dataset)):
      
      # Get image 3d
      subject_dict = {}
      ls_modality = ['FLAIR', 'T1w', 'T1wCE', 'T2w']
      for modality in ls_modality:
        img_3d = self.dataset[patient_index][modality]['data'][0]
            
        # Get bbox
        ls_min_uni_img = []
        for i in range(img_3d.shape[2]):
          img_slice = img_3d[:,:,i]
          uni_min_slice = torch.where(img_slice<0, np.abs(img_slice), img_slice)
          ls_min_uni_img.append(uni_min_slice)  
        
        # Get net img 3d with minimum value at 0
        new_img_3d = torch.stack(ls_min_uni_img, dim=2)

        # Get black area of each slice
        ls_non_black = []
        for i in range(new_img_3d.shape[2]):
          img_slice = new_img_3d[:,:,i]
          non_black_area = torch.sum(img_slice > 0).numpy().item()
          ls_non_black.append(non_black_area)

        ls_slice_remain = [idx for idx,n in enumerate(ls_non_black) if n > min_area_threshold]
        z_lower, z_higher = ls_slice_remain[0], ls_slice_remain[-1]

        # Get largest slice
        index_largest_slice = np.argmax(ls_non_black)
        largest_slice = new_img_3d[:,:,index_largest_slice]

        # Get bbox
        ls_col_x = []
        for i,col in enumerate(largest_slice):
          if torch.max(col) > 0:
            ls_col_x.append(i)

        ls_col_y = []
        for i,col in enumerate(largest_slice.T):
          if torch.max(col) > 0:
            ls_col_y.append(i)

        img_arr_3d_cropped = new_img_3d[ls_col_x, :, :]
        img_arr_3d_cropped = img_arr_3d_cropped[:, ls_col_y, :]
        img_arr_3d_cropped = img_arr_3d_cropped[:,:, z_lower:z_higher]

        # Get an extra dim as the channel
        img_arr_3d_cropped = img_arr_3d_cropped.unsqueeze(dim=0)

        # Rescale back to (64,64,64)
        resize = tio.transforms.Resize((64,64,64))
        img_arr_3d_cropped = resize(img_arr_3d_cropped)

        # Append to list 
        scalar_image_temp = tio.ScalarImage(tensor=img_arr_3d_cropped)
        subject_dict[modality] = scalar_image_temp

      subject_dict['BraTS21ID'] = self.dataset[patient_index]['BraTS21ID']
      subject_dict['MGMT_value'] = self.dataset[patient_index]['MGMT_value']
      subject_new = tio.Subject(subject_dict)
      self.ls_new_subjects.append(subject_new)

    train_set = tio.SubjectsDataset(self.ls_new_subjects, 
                                    transform=self.transform_phase_2)

# Training mode: 90/10% split
    split_ratio = 0.9
    n_train_set = int(len(train_set) * 0.9)
    n_val_set = len(train_set) - n_train_set

    self.train_set, self.val_set = random_split(train_set,
                                                [n_train_set, n_val_set],
                                                generator=torch.Generator().manual_seed(1))
        
    
  def train_dataloader(self):
    return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)

  def val_dataloader(self):
    return DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False)
  
# call main
if __name__ == '__main__':
  dataset = RSNAPre(data_dir='/content/drive/MyDrive/Data/rsna_practice')
  dataset.setup()
  train_loader = dataset.train_dataloader()
  val_loader = dataset.val_dataloader()